<a href="https://colab.research.google.com/github/Etuarda/EstudosTSMinasProgramam25-2/blob/main/notebook_semana01_pnl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega 1 - Pré-processamento e Análise Morfossintática

**Curso:** Processamento de Linguagem Natural  
**Aluno:** Eduarda Silva Santos
**Data:** 16/01/2026

---

## Objetivo

Implementar um **pipeline de pré-processamento e análise morfossintática** que será reutilizado no projeto final do chatbot de e-commerce.

## Instruções

1. Complete todos os exercícios marcados com `# === SEU CÓDIGO AQUI ===`
2. **Não modifique** os nomes das funções ou variáveis indicadas
3. Execute todas as células em ordem antes de submeter
4. As funções implementadas serão reutilizadas nas próximas entregas

## Critérios de Avaliação

| Questão | Pontos | Descrição |
|---------|--------|------------|
| Q1 | 8 | Limpeza básica de texto |
| Q2 | 8 | Tokenização com NLTK |
| Q3 | 8 | Remoção de stopwords |
| Q4 | 8 | Stemming RSLP |
| Q5 | 10 | Pipeline completo |
| Q6 | 10 | Análise de frequência |
| Q7 | 10 | Processamento em lote |
| Q8 | 8 | Extração de POS tags (spaCy) |
| Q9 | 8 | Extração de substantivos (spaCy) |
| Q10 | 7 | Extração de adjetivos (spaCy) |
| Q11 | 7 | Extração de verbos (spaCy) |
| Q12 | 8 | Função integrada final |
| **Total** | **100** | |

---

## Dataset

Utilizaremos o dataset **B2W-Reviews01** com reviews de e-commerce brasileiro.

---

## Setup Inicial

Execute as células abaixo para configurar o ambiente.

In [ ]:
# === INSTALAÇÃO DE DEPENDÊNCIAS ===
# Não modifique esta célula

!pip install nltk pandas numpy matplotlib wordcloud spacy --quiet
!python -m spacy download pt_core_news_sm --quiet

import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('rslp', quiet=True)
nltk.download('punkt_tab', quiet=True)

print("Dependências instaladas!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 32.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Dependências instaladas!


In [ ]:
# === IMPORTS ===
# Não modifique esta célula

import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

# NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

# spaCy
import spacy
nlp_spacy = spacy.load('pt_core_news_sm')

# Configurações
import warnings
warnings.filterwarnings('ignore')

# Constantes do exercício
SEED_AVALIACAO = 42
np.random.seed(SEED_AVALIACAO)

print("Imports realizados!")
print(f"NLTK versão: {nltk.__version__}")
print(f"spaCy versão: {spacy.__version__}")

Imports realizados!
NLTK versão: 3.9.1
spaCy versão: 3.8.11


In [ ]:
# === CARREGAR DATASET ===
# Não modifique esta célula

URL_DATASET_B2W = "https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/main/B2W-Reviews01.csv"

df_reviews_b2w = pd.read_csv(URL_DATASET_B2W, nrows=1000)
df_reviews_b2w = df_reviews_b2w.dropna(subset=['review_text'])
df_reviews_b2w = df_reviews_b2w.reset_index(drop=True)

print(f"Dataset carregado: {len(df_reviews_b2w)} reviews")
print(f"Colunas: {list(df_reviews_b2w.columns)}")

# Exibir amostra
df_reviews_b2w[['review_text', 'overall_rating']].head(3)

Dataset carregado: 980 reviews
Colunas: ['submission_date', 'reviewer_id', 'product_id', 'product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2', 'review_title', 'overall_rating', 'recommend_to_a_friend', 'review_text', 'reviewer_birth_year', 'reviewer_gender', 'reviewer_state']


,review_text,overall_rating
0,Estou contente com a compra entrega rápida o ú...,4
1,"Por apenas R$1994.20,eu consegui comprar esse ...",4
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,4


In [ ]:
# === AMOSTRAS PARA EXERCÍCIOS ===
# Não modifique esta célula

# Textos específicos para validação (NLTK)
TEXTO_VALIDACAO_Q1 = df_reviews_b2w['review_text'].iloc[7]
TEXTO_VALIDACAO_Q2 = df_reviews_b2w['review_text'].iloc[13]
TEXTO_VALIDACAO_Q5 = df_reviews_b2w['review_text'].iloc[21]

# Índices para processamento em lote
INDICES_LOTE_Q7 = [3, 8, 15, 22, 31, 47, 56, 63, 78, 89]
REVIEWS_LOTE_Q7 = df_reviews_b2w['review_text'].iloc[INDICES_LOTE_Q7].tolist()

# Textos específicos para validação (spaCy)
TEXTO_VALIDACAO_Q8 = df_reviews_b2w['review_text'].iloc[17]
TEXTO_VALIDACAO_Q9 = df_reviews_b2w['review_text'].iloc[29]
TEXTO_VALIDACAO_Q10 = df_reviews_b2w['review_text'].iloc[41]
TEXTO_VALIDACAO_Q11 = df_reviews_b2w['review_text'].iloc[53]

print("Amostras de validação definidas!")
print(f"\nTextos NLTK:")
print(f"  Q1 (índice 7): '{TEXTO_VALIDACAO_Q1[:80]}...'")
print(f"  Q2 (índice 13): '{TEXTO_VALIDACAO_Q2[:80]}...'")
print(f"\nTextos spaCy:")
print(f"  Q8 (índice 17): '{TEXTO_VALIDACAO_Q8[:80]}...'")
print(f"  Q9 (índice 29): '{TEXTO_VALIDACAO_Q9[:80]}...'")

Amostras de validação definidas!

Textos NLTK:
  Q1 (índice 7): 'Produto excelente qualidade boa câmera desenvolvimento do Android com rapidez...'
  Q2 (índice 13): 'Esse celular não vale nada a bateria não vale nada descarrega sozinha quando est...'

Textos spaCy:
  Q8 (índice 17): 'nao tenho nada a reclamar, bom atendimento e entrega dentro do combinado....'
  Q9 (índice 29): 'Uma tela impecável. Se sua prioridade é tela e câmera, esse é o cara. O ponto fr...'


---

# PARTE 1: Pré-processamento com NLTK (54 pontos)

Nesta parte, você implementará funções de pré-processamento usando NLTK e regex.

---

## Questão 1: Limpeza Básica de Texto (8 pontos)

Implemente a função `limpar_texto` que realiza as seguintes operações:

1. Converte o texto para **minúsculas**
2. Remove **URLs** (padrões http://, https://, www.)
3. Remove **emails** (padrões com @)
4. Remove **números** isolados
5. Remove **pontuação** excessiva (mantendo espaços)
6. Remove **espaços múltiplos** (substituir por espaço único)
7. Remove espaços no **início e fim** do texto

**Parâmetros:**
- `texto` (str): Texto a ser limpo

**Retorno:**
- `str`: Texto limpo

**Dica:** Use expressões regulares (`re.sub`).

In [ ]:
def limpar_texto(texto: str) -> str:
    """
    Realiza limpeza básica de texto para pré-processamento.

    Operações:
    - Converte para minúsculas
    - Remove URLs, emails, números isolados
    - Remove pontuação e espaços extras

    Parâmetros:
        texto (str): Texto original

    Retorna:
        str: Texto limpo

    Exemplo:
        >>> limpar_texto("PRODUTO BOM!!! Visite http://loja.com")
        'produto bom visite'
    """
    if texto is None:
        return ""

    # 1) minúsculas
    texto = str(texto).lower()

    # 2) remover URLs (http://, https://, www.)
    texto = re.sub(r"\bhttps?://\S+\b", " ", texto)
    texto = re.sub(r"\bwww\.\S+\b", " ", texto)

    # 3) remover emails (com @)
    texto = re.sub(r"\b[\w\.-]+@[\w\.-]+\.\w+\b", " ", texto)

    # 4) remover números isolados
    texto = re.sub(r"\b\d+\b", " ", texto)

    # 5) remover pontuação (mantendo espaços)
    texto = re.sub(rf"[{re.escape(string.punctuation)}]", " ", texto)

    # 6) remover espaços múltiplos (virar um só) + 7) trim
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto


# === APLICAR NO TEXTO DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

texto_limpo_q1 = limpar_texto(TEXTO_VALIDACAO_Q1)
num_palavras_q1 = len(texto_limpo_q1.split())


In [ ]:
# Validação da Q1 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q1: limpar_texto")
print("=" * 60)
print(f"\nTexto original (índice 7):")
print(f"  '{TEXTO_VALIDACAO_Q1}'")
print(f"\nTexto limpo:")
print(f"  '{texto_limpo_q1}'")
print(f"\nEstatísticas:")
print(f"  Palavras: {num_palavras_q1}")
print(f"  Tamanho: {len(texto_limpo_q1)} caracteres")

VALIDAÇÃO Q1: limpar_texto

Texto original (índice 7):
  'Produto excelente qualidade boa câmera desenvolvimento do Android com rapidez'

Texto limpo:
  'produto excelente qualidade boa câmera desenvolvimento do android com rapidez'

Estatísticas:
  Palavras: 10
  Tamanho: 77 caracteres


---

## Questão 2: Tokenização com NLTK (8 pontos)

Implemente a função `tokenizar` que divide o texto em tokens (palavras).

**Requisitos:**
1. Use `word_tokenize` do NLTK com `language='portuguese'`
2. Retorne apenas tokens **alfabéticos** (sem números ou pontuação)
3. Tokens devem estar em **minúsculas**

**Parâmetros:**
- `texto` (str): Texto a ser tokenizado

**Retorno:**
- `list[str]`: Lista de tokens

**Dica:** Use `str.isalpha()` para verificar se token é alfabético.

In [ ]:
def tokenizar(texto: str) -> list:
    """
    Tokeniza texto usando NLTK word_tokenize.

    Retorna apenas tokens alfabéticos em minúsculas.

    Parâmetros:
        texto (str): Texto a tokenizar

    Retorna:
        list: Lista de tokens (palavras)

    Exemplo:
        >>> tokenizar("Produto bom! Entrega rápida.")
        ['produto', 'bom', 'entrega', 'rápida']
    """
    if texto is None:
        return []

    # Tokenização com NLTK (português)
    tokens = word_tokenize(str(texto), language="portuguese")

    # Manter apenas tokens alfabéticos e converter para minúsculas
    tokens = [token.lower() for token in tokens if token.isalpha()]

    return tokens


# === APLICAR NO TEXTO DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

tokens_q2 = tokenizar(TEXTO_VALIDACAO_Q2)
num_tokens_q2 = len(tokens_q2)


In [ ]:
# Validação da Q2 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q2: tokenizar")
print("=" * 60)
print(f"\nTexto original (índice 13):")
print(f"  '{TEXTO_VALIDACAO_Q2}'")
print(f"\nTokens gerados ({num_tokens_q2} tokens):")
print(f"  {tokens_q2}")
print(f"\nPrimeiros 10 tokens:")
print(f"  {tokens_q2[:10]}")

VALIDAÇÃO Q2: tokenizar

Texto original (índice 13):
  'Esse celular não vale nada a bateria não vale nada descarrega sozinha quando estar usando esquenta tanto que faz medo espludi foi a pior compra da minha vida se eu tivesse condições quebrava ele em mil pedaços e comprava outro de qualquer marca menos Motorola'

Tokens gerados (45 tokens):
  ['esse', 'celular', 'não', 'vale', 'nada', 'a', 'bateria', 'não', 'vale', 'nada', 'descarrega', 'sozinha', 'quando', 'estar', 'usando', 'esquenta', 'tanto', 'que', 'faz', 'medo', 'espludi', 'foi', 'a', 'pior', 'compra', 'da', 'minha', 'vida', 'se', 'eu', 'tivesse', 'condições', 'quebrava', 'ele', 'em', 'mil', 'pedaços', 'e', 'comprava', 'outro', 'de', 'qualquer', 'marca', 'menos', 'motorola']

Primeiros 10 tokens:
  ['esse', 'celular', 'não', 'vale', 'nada', 'a', 'bateria', 'não', 'vale', 'nada']


---

## Questão 3: Remoção de Stopwords (8 pontos)

Implemente a função `remover_stopwords` que remove palavras comuns (stopwords) de uma lista de tokens.

**Requisitos:**
1. Use a lista de stopwords do NLTK para português
2. Mantenha apenas tokens que **não** estão na lista de stopwords
3. Retorne os tokens na mesma ordem

**Parâmetros:**
- `tokens` (list): Lista de tokens

**Retorno:**
- `list[str]`: Lista de tokens sem stopwords

**Dica:** Use `stopwords.words('portuguese')`

In [ ]:
def remover_stopwords(tokens: list) -> list:
    """
    Remove stopwords (palavras comuns) de lista de tokens.

    Usa lista de stopwords do NLTK para português.

    Parâmetros:
        tokens (list): Lista de tokens

    Retorna:
        list: Lista de tokens sem stopwords

    Exemplo:
        >>> remover_stopwords(['o', 'produto', 'é', 'bom'])
        ['produto', 'bom']
    """
    if not tokens:
        return []

    # Lista de stopwords em português
    stopwords_pt = set(stopwords.words('portuguese'))

    # Manter apenas tokens que não são stopwords, preservando a ordem
    tokens_filtrados = [token for token in tokens if token not in stopwords_pt]

    return tokens_filtrados


# === APLICAR NOS TOKENS DA Q2 ===
# Não modifique os nomes das variáveis abaixo

tokens_sem_stopwords_q3 = remover_stopwords(tokens_q2)
num_tokens_removidos_q3 = num_tokens_q2 - len(tokens_sem_stopwords_q3)


In [ ]:
# Validação da Q3 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q3: remover_stopwords")
print("=" * 60)
print(f"\nTokens originais ({num_tokens_q2}):")
print(f"  {tokens_q2[:15]}...")
print(f"\nTokens sem stopwords ({len(tokens_sem_stopwords_q3)}):")
print(f"  {tokens_sem_stopwords_q3[:15]}...")
print(f"\nEstatísticas:")
print(f"  Tokens removidos: {num_tokens_removidos_q3}")
print(f"  Taxa de remoção: {num_tokens_removidos_q3/num_tokens_q2*100:.1f}%")

VALIDAÇÃO Q3: remover_stopwords

Tokens originais (45):
  ['esse', 'celular', 'não', 'vale', 'nada', 'a', 'bateria', 'não', 'vale', 'nada', 'descarrega', 'sozinha', 'quando', 'estar', 'usando']...

Tokens sem stopwords (27):
  ['celular', 'vale', 'nada', 'bateria', 'vale', 'nada', 'descarrega', 'sozinha', 'usando', 'esquenta', 'tanto', 'faz', 'medo', 'espludi', 'pior']...

Estatísticas:
  Tokens removidos: 18
  Taxa de remoção: 40.0%


---

## Questão 4: Stemming com RSLP (8 pontos)

Implemente a função `aplicar_stemming` que reduz palavras à sua raiz usando o algoritmo RSLP.

**Requisitos:**
1. Use `RSLPStemmer` do NLTK
2. Aplique stemming em cada token da lista
3. Retorne lista de tokens "stemmizados"

**Parâmetros:**
- `tokens` (list): Lista de tokens

**Retorno:**
- `list[str]`: Lista de tokens com stemming aplicado

**Dica:** RSLP é específico para português brasileiro.

In [ ]:
def aplicar_stemming(tokens: list) -> list:
    """
    Aplica stemming RSLP em lista de tokens.

    Reduz cada palavra à sua raiz morfológica.

    Parâmetros:
        tokens (list): Lista de tokens

    Retorna:
        list: Lista de tokens com stemming

    Exemplo:
        >>> aplicar_stemming(['produtos', 'comprando', 'vendedores'])
        ['produt', 'compr', 'vended']
    """
    if not tokens:
        return []

    # Inicializa o stemmer RSLP (português brasileiro)
    stemmer = RSLPStemmer()

    # Aplica stemming em cada token
    tokens_stemmizados = [stemmer.stem(token) for token in tokens]

    return tokens_stemmizados


# === APLICAR NOS TOKENS SEM STOPWORDS ===
# Não modifique os nomes das variáveis abaixo

tokens_stemming_q4 = aplicar_stemming(tokens_sem_stopwords_q3)
tokens_unicos_q4 = len(set(tokens_stemming_q4))


In [ ]:
# Validação da Q4 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q4: aplicar_stemming")
print("=" * 60)
print(f"\nTokens antes ({len(tokens_sem_stopwords_q3)} tokens):")
print(f"  {tokens_sem_stopwords_q3[:10]}")
print(f"\nTokens após stemming ({len(tokens_stemming_q4)} tokens):")
print(f"  {tokens_stemming_q4[:10]}")
print(f"\nEstatísticas:")
print(f"  Tokens únicos antes: {len(set(tokens_sem_stopwords_q3))}")
print(f"  Tokens únicos após: {tokens_unicos_q4}")
print(f"  Redução de vocabulário: {len(set(tokens_sem_stopwords_q3)) - tokens_unicos_q4} tokens")

VALIDAÇÃO Q4: aplicar_stemming

Tokens antes (27 tokens):
  ['celular', 'vale', 'nada', 'bateria', 'vale', 'nada', 'descarrega', 'sozinha', 'usando', 'esquenta']

Tokens após stemming (27 tokens):
  ['celul', 'val', 'nad', 'bat', 'val', 'nad', 'descarreg', 'so', 'us', 'esquent']

Estatísticas:
  Tokens únicos antes: 25
  Tokens únicos após: 24
  Redução de vocabulário: 1 tokens


---

## Questão 5: Pipeline Completo de Pré-processamento (10 pontos)

Implemente a função `preprocessar_texto` que combina todas as etapas anteriores em um pipeline único.

**Pipeline:**
1. Limpeza (`limpar_texto`)
2. Tokenização (`tokenizar`)
3. Remoção de stopwords (`remover_stopwords`)
4. Stemming (`aplicar_stemming`)
5. Juntar tokens em string (separados por espaço)

**IMPORTANTE:** Você DEVE usar as funções implementadas nas questões anteriores.

**Parâmetros:**
- `texto` (str): Texto original

**Retorno:**
- `str`: Texto pré-processado

In [ ]:
def preprocessar_texto(texto: str) -> str:
    """
    Pipeline completo de pré-processamento.

    Aplica todas as etapas: limpeza, tokenização,
    remoção de stopwords e stemming.

    Parâmetros:
        texto (str): Texto original

    Retorna:
        str: Texto pré-processado

    Exemplo:
        >>> preprocessar_texto("Os produtos chegaram rápido!")
        'produt cheg rapid'
    """
    if texto is None:
        return ""

    # 1) Limpeza básica
    texto_limpo = limpar_texto(texto)

    # 2) Tokenização
    tokens = tokenizar(texto_limpo)

    # 3) Remoção de stopwords
    tokens_sem_stopwords = remover_stopwords(tokens)

    # 4) Stemming
    tokens_stemming = aplicar_stemming(tokens_sem_stopwords)

    # 5) Juntar tokens em uma string
    texto_preprocessado = " ".join(tokens_stemming)

    return texto_preprocessado


# === APLICAR NO TEXTO DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

texto_preprocessado_q5 = preprocessar_texto(TEXTO_VALIDACAO_Q5)
num_termos_finais_q5 = len(texto_preprocessado_q5.split())


In [ ]:
# Validação da Q5 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q5: preprocessar_texto")
print("=" * 60)
print(f"\nTexto original (índice 21):")
print(f"  '{TEXTO_VALIDACAO_Q5}'")
print(f"\nTexto pré-processado:")
print(f"  '{texto_preprocessado_q5}'")
print(f"\nEstatísticas:")
print(f"  Termos finais: {num_termos_finais_q5}")
print(f"  Tamanho original: {len(TEXTO_VALIDACAO_Q5)} caracteres")
print(f"  Tamanho processado: {len(texto_preprocessado_q5)} caracteres")
print(f"  Redução: {(1 - len(texto_preprocessado_q5)/len(TEXTO_VALIDACAO_Q5))*100:.1f}%")

VALIDAÇÃO Q5: preprocessar_texto

Texto original (índice 21):
  'Pontos Positivos Painel de 10bits (isso faz toda diferença da mais de 1 bilhão de cores) Bluetooth Wi-Fi AC USB 3.0 Input Lag Baixo Painel VA, que proporciona um alto contraste. Android TV 6.0 Som da TV realmente é excepcional! (saída do auto falantes são voltados para frente) Vem 2 controles remotos  A tv realmente é formidável, valeu muito a pena ter comprado TCL.'

Texto pré-processado:
  'pont posi painel faz tod diferenç bilhã cor bluetooth wi fi ac usb input lag baix painel va proporc alt contr android tv som tv real excepc saíd aut fal volt frent vem control remot tv real formid val pen ter compr tcl'

Estatísticas:
  Termos finais: 43
  Tamanho original: 368 caracteres
  Tamanho processado: 218 caracteres
  Redução: 40.8%


---

## Questão 6: Análise de Frequência de Termos (10 pontos)

Implemente a função `analisar_frequencia` que retorna os N termos mais frequentes em um texto.

**Requisitos:**
1. Pré-processe o texto usando `preprocessar_texto`
2. Conte a frequência de cada termo
3. Retorne lista de tuplas `(termo, frequencia)` ordenada por frequência (maior → menor)

**Parâmetros:**
- `texto` (str): Texto a analisar
- `n` (int): Número de termos mais frequentes a retornar (padrão: 10)

**Retorno:**
- `list[tuple]`: Lista de (termo, frequencia)

**Dica:** Use `Counter` do módulo `collections`.

In [ ]:
def analisar_frequencia(texto: str, n: int = 10) -> list:
    """
    Retorna os N termos mais frequentes em um texto.

    Parâmetros:
        texto (str): Texto a analisar
        n (int): Número de termos a retornar

    Retorna:
        list: Lista de tuplas (termo, frequencia)

    Exemplo:
        >>> analisar_frequencia("produto produto bom bom bom", n=2)
        [('bom', 3), ('produto', 2)]
    """
    if texto is None:
        return []

    # 1) Pré-processar o texto (pipeline completo)
    texto_preprocessado = preprocessar_texto(texto)

    if not texto_preprocessado:
        return []

    # 2) Quebrar em termos
    termos = texto_preprocessado.split()

    # 3) Contar frequência
    contagem = Counter(termos)

    # 4) Retornar os N termos mais frequentes (ordem decrescente)
    return contagem.most_common(n)


# === APLICAR EM MÚLTIPLOS REVIEWS ===
# Não modifique os nomes das variáveis abaixo

# Concatenar primeiros 50 reviews
texto_analise_q6 = " ".join(df_reviews_b2w['review_text'].iloc[:50].tolist())
termos_frequentes_q6 = analisar_frequencia(texto_analise_q6, n=15)
termo_mais_frequente_q6 = termos_frequentes_q6[0][0] if termos_frequentes_q6 else None


In [ ]:
# Validação da Q6 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q6: analisar_frequencia")
print("=" * 60)
print(f"\nAnálise de 50 primeiros reviews")
print(f"\nTop 15 termos mais frequentes:")
for i, (termo, freq) in enumerate(termos_frequentes_q6, 1):
    print(f"  {i:2d}. '{termo}' → {freq} ocorrências")
print(f"\nTermo mais frequente: '{termo_mais_frequente_q6}'")

VALIDAÇÃO Q6: analisar_frequencia

Análise de 50 primeiros reviews

Top 15 termos mais frequentes:
   1. 'produt' → 22 ocorrências
   2. 'compr' → 16 ocorrências
   3. 'entreg' → 13 ocorrências
   4. 'recom' → 9 ocorrências
   5. 'gost' → 9 ocorrências
   6. 'sup' → 8 ocorrências
   7. 'praz' → 8 ocorrências
   8. 'excel' → 8 ocorrências
   9. 'americ' → 7 ocorrências
  10. 'nao' → 7 ocorrências
  11. 'outr' → 6 ocorrências
  12. 'bom' → 6 ocorrências
  13. 'câm' → 6 ocorrências
  14. 'bem' → 6 ocorrências
  15. 'parabém' → 5 ocorrências

Termo mais frequente: 'produt'


---

## Questão 7: Processamento em Lote (10 pontos)

Implemente a função `processar_lote` que processa múltiplos textos de uma vez.

**Requisitos:**
1. Receba uma lista de textos
2. Aplique `preprocessar_texto` em cada texto
3. Retorne lista de textos pré-processados na mesma ordem

**Parâmetros:**
- `lista_textos` (list): Lista de textos originais

**Retorno:**
- `list[str]`: Lista de textos pré-processados

In [ ]:
def processar_lote(lista_textos: list) -> list:
    """
    Processa múltiplos textos em lote.

    Aplica preprocessar_texto em cada texto da lista.

    Parâmetros:
        lista_textos (list): Lista de textos originais

    Retorna:
        list: Lista de textos pré-processados

    Exemplo:
        >>> processar_lote(["Texto 1", "Texto 2"])
        ['text', 'text']
    """
    if not lista_textos:
        return []

    # Aplicar o pipeline de pré-processamento em cada texto, mantendo a ordem
    textos_processados = [preprocessar_texto(texto) for texto in lista_textos]

    return textos_processados


# === APLICAR NO LOTE DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

reviews_processadas_q7 = processar_lote(REVIEWS_LOTE_Q7)

# Calcular média de termos por review
num_termos_por_review = [len(texto.split()) for texto in reviews_processadas_q7]
media_termos_q7 = sum(num_termos_por_review) / len(num_termos_por_review)


In [ ]:
# Validação da Q7 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q7: processar_lote")
print("=" * 60)

# Verificar lote de entrada
print(f"\nLote de entrada:")
print(f"  Número de reviews: {len(REVIEWS_LOTE_Q7)}")
print(f"  Índices: {INDICES_LOTE_Q7}")

# Verificar saída
print(f"\nResultados:")
print(f"  Reviews processadas: {len(reviews_processadas_q7)}")
print(f"  Média de termos/review: {media_termos_q7:.2f}")

# Exibir amostra
print(f"\nAmostra (3 primeiras):")
for i in range(3):
    original = REVIEWS_LOTE_Q7[i][:60] + "..."
    processado = reviews_processadas_q7[i][:60] + "..."
    print(f"\n  [{INDICES_LOTE_Q7[i]}] Original:   '{original}'")
    print(f"      Processado: '{processado}'")

VALIDAÇÃO Q7: processar_lote

Lote de entrada:
  Número de reviews: 10
  Índices: [3, 8, 15, 22, 31, 47, 56, 63, 78, 89]

Resultados:
  Reviews processadas: 10
  Média de termos/review: 9.20

Amostra (3 primeiras):

  [3] Original:   'MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS DETALHES QUE TÊ...'
      Processado: 'filh amou parec verdad tant detalh têm...'

  [8] Original:   'O barulho e minimo e o vento é bem forte na velocidade 2...'
      Processado: 'barulh min vent bem fort veloc...'

  [15] Original:   'a mochila nao esta fechando direito por isso nao recomendo s...'
      Processado: 'mochil nao fech direit nao recom filh nao deix suj ia devolv...'


---

# PARTE 2: Análise Morfossintática com spaCy (30 pontos)

Nesta parte, você implementará funções de análise linguística usando spaCy.

---

## Questão 8: Extração de POS Tags (8 pontos)

Implemente a função `extrair_pos_tags` que retorna os POS tags (classes gramaticais) de cada token.

**Requisitos:**
1. Use o modelo spaCy `nlp_spacy` (já carregado)
2. Para cada token, extraia: texto, POS tag e lema
3. Retorne lista de tuplas `(texto, pos, lema)`

**Parâmetros:**
- `texto` (str): Texto a analisar

**Retorno:**
- `list[tuple]`: Lista de (texto, pos, lema)

**Dica:** Use `token.text`, `token.pos_`, `token.lemma_`

In [ ]:
def extrair_pos_tags(texto: str) -> list:
    """
    Extrai POS tags de cada token usando spaCy.

    Parâmetros:
        texto (str): Texto a analisar

    Retorna:
        list: Lista de tuplas (texto, pos, lema)

    Exemplo:
        >>> extrair_pos_tags("O produto é bom")
        [('O', 'DET', 'o'), ('produto', 'NOUN', 'produto'), ...]
    """
    if texto is None:
        return []

    # Processar texto com o modelo spaCy já carregado
    doc = nlp_spacy(str(texto))

    # Extrair (texto, POS tag, lema) de cada token
    pos_tags = [(token.text, token.pos_, token.lemma_) for token in doc]

    return pos_tags


# === APLICAR NO TEXTO DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

pos_tags_q8 = extrair_pos_tags(TEXTO_VALIDACAO_Q8)
num_tokens_q8 = len(pos_tags_q8)

# Contar distribuição de POS
pos_counts_q8 = Counter([tag[1] for tag in pos_tags_q8])
pos_mais_frequente_q8 = pos_counts_q8.most_common(1)[0][0] if pos_counts_q8 else None


In [ ]:
# Validação da Q8 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q8: extrair_pos_tags")
print("=" * 60)
print(f"\nTexto original (índice 17):")
print(f"  '{TEXTO_VALIDACAO_Q8}'")
print(f"\nPOS tags extraídos ({num_tokens_q8} tokens):")
for i, (texto, pos, lema) in enumerate(pos_tags_q8[:10]):
    print(f"  {i+1:2d}. '{texto}' → POS: {pos:6s} | Lema: '{lema}'")
print(f"\nDistribuição de POS:")
for pos, count in pos_counts_q8.most_common(5):
    print(f"  {pos}: {count} ({count/num_tokens_q8*100:.1f}%)")
print(f"\nPOS mais frequente: {pos_mais_frequente_q8}")

VALIDAÇÃO Q8: extrair_pos_tags

Texto original (índice 17):
  'nao tenho nada a reclamar, bom atendimento e entrega dentro do combinado.'

POS tags extraídos (14 tokens):
   1. 'nao' → POS: SCONJ  | Lema: 'nao'
   2. 'tenho' → POS: VERB   | Lema: 'ter'
   3. 'nada' → POS: PRON   | Lema: 'nada'
   4. 'a' → POS: SCONJ  | Lema: 'a'
   5. 'reclamar' → POS: VERB   | Lema: 'reclamar'
   6. ',' → POS: PUNCT  | Lema: ','
   7. 'bom' → POS: ADJ    | Lema: 'bom'
   8. 'atendimento' → POS: NOUN   | Lema: 'atendimento'
   9. 'e' → POS: CCONJ  | Lema: 'e'
  10. 'entrega' → POS: VERB   | Lema: 'entregar'

Distribuição de POS:
  VERB: 3 (21.4%)
  SCONJ: 2 (14.3%)
  PUNCT: 2 (14.3%)
  NOUN: 2 (14.3%)
  PRON: 1 (7.1%)

POS mais frequente: VERB


---

## Questão 9: Extração de Substantivos (8 pontos)

Implemente a função `extrair_substantivos` que retorna apenas os substantivos (NOUN) de um texto.

**Requisitos:**
1. Use o modelo spaCy `nlp_spacy`
2. Filtre tokens com `pos_ == 'NOUN'`
3. Retorne os **lemas** dos substantivos (forma canônica)
4. Converta para minúsculas

**Parâmetros:**
- `texto` (str): Texto a analisar

**Retorno:**
- `list[str]`: Lista de lemas dos substantivos

In [ ]:
def extrair_substantivos(texto: str) -> list:
    """
    Extrai substantivos (lemas) de um texto.

    Parâmetros:
        texto (str): Texto a analisar

    Retorna:
        list: Lista de lemas dos substantivos

    Exemplo:
        >>> extrair_substantivos("Os produtos chegaram ontem")
        ['produto']
    """
    if texto is None:
        return []

    # Processar texto com spaCy
    doc = nlp_spacy(str(texto))

    # Filtrar apenas substantivos (NOUN), retornar lemas em minúsculas
    substantivos = [token.lemma_.lower() for token in doc if token.pos_ == "NOUN"]

    return substantivos


# === APLICAR NO TEXTO DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

substantivos_q9 = extrair_substantivos(TEXTO_VALIDACAO_Q9)
num_substantivos_q9 = len(substantivos_q9)
substantivos_unicos_q9 = len(set(substantivos_q9))


In [ ]:
# Validação da Q9 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q9: extrair_substantivos")
print("=" * 60)
print(f"\nTexto original (índice 29):")
print(f"  '{TEXTO_VALIDACAO_Q9}'")
print(f"\nSubstantivos extraídos ({num_substantivos_q9}):")
print(f"  {substantivos_q9}")
print(f"\nEstatísticas:")
print(f"  Total de substantivos: {num_substantivos_q9}")
print(f"  Substantivos únicos: {substantivos_unicos_q9}")
if substantivos_q9:
    freq_subst = Counter(substantivos_q9)
    print(f"\nSubstantivos mais frequentes:")
    for subst, count in freq_subst.most_common(5):
        print(f"  '{subst}': {count}")

VALIDAÇÃO Q9: extrair_substantivos

Texto original (índice 29):
  'Uma tela impecável. Se sua prioridade é tela e câmera, esse é o cara. O ponto fraco fica por conta do armazenamento, apenas 16GB, e pela câmera frontal, de apenas 5Mpixels sem flash. No entanto, a câmera principal de 13Mpixels é perfeita.'

Substantivos extraídos (10):
  ['tela', 'prioridade', 'cara', 'ponto', 'conta', 'armazenamento', 'câmera', 'flash', 'entanto', 'câmera']

Estatísticas:
  Total de substantivos: 10
  Substantivos únicos: 9

Substantivos mais frequentes:
  'câmera': 2
  'tela': 1
  'prioridade': 1
  'cara': 1
  'ponto': 1


---

## Questão 10: Extração de Adjetivos (7 pontos)

Implemente a função `extrair_adjetivos` que retorna apenas os adjetivos (ADJ) de um texto.

**Requisitos:**
1. Use o modelo spaCy `nlp_spacy`
2. Filtre tokens com `pos_ == 'ADJ'`
3. Retorne os **lemas** dos adjetivos
4. Converta para minúsculas

**Parâmetros:**
- `texto` (str): Texto a analisar

**Retorno:**
- `list[str]`: Lista de lemas dos adjetivos

**Dica:** Adjetivos são importantes para análise de sentimento!

In [ ]:
def extrair_adjetivos(texto: str) -> list:
    """
    Extrai adjetivos (lemas) de um texto.

    Parâmetros:
        texto (str): Texto a analisar

    Retorna:
        list: Lista de lemas dos adjetivos

    Exemplo:
        >>> extrair_adjetivos("O produto é muito bom e rápido")
        ['bom', 'rápido']
    """
    if texto is None:
        return []

    # Processar texto com o modelo spaCy
    doc = nlp_spacy(str(texto))

    # Filtrar apenas adjetivos (ADJ), retornar lemas em minúsculas
    adjetivos = [token.lemma_.lower() for token in doc if token.pos_ == "ADJ"]

    return adjetivos


# === APLICAR NO TEXTO DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

adjetivos_q10 = extrair_adjetivos(TEXTO_VALIDACAO_Q10)
num_adjetivos_q10 = len(adjetivos_q10)


In [ ]:
# Validação da Q10 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q10: extrair_adjetivos")
print("=" * 60)
print(f"\nTexto original (índice 41):")
print(f"  '{TEXTO_VALIDACAO_Q10}'")
print(f"\nAdjetivos extraídos ({num_adjetivos_q10}):")
print(f"  {adjetivos_q10}")
print(f"\nEstatísticas:")
print(f"  Total de adjetivos: {num_adjetivos_q10}")
print(f"  Adjetivos únicos: {len(set(adjetivos_q10))}")

VALIDAÇÃO Q10: extrair_adjetivos

Texto original (índice 41):
  'Fiquei satisfeito com a segurança e garantia de toda transação, entretanto fiz outra compra no dia 29/11 e até hoje não foi entregue. O que diferencia uma compra da outra com relação a entrega?'

Adjetivos extraídos (0):
  []

Estatísticas:
  Total de adjetivos: 0
  Adjetivos únicos: 0


---

## Questão 11: Extração de Verbos (7 pontos)

Implemente a função `extrair_verbos` que retorna apenas os verbos (VERB) de um texto.

**Requisitos:**
1. Use o modelo spaCy `nlp_spacy`
2. Filtre tokens com `pos_ == 'VERB'`
3. Retorne os **lemas** dos verbos (forma infinitiva)
4. Converta para minúsculas

**Parâmetros:**
- `texto` (str): Texto a analisar

**Retorno:**
- `list[str]`: Lista de lemas dos verbos

In [ ]:
def extrair_verbos(texto: str) -> list:
    """
    Extrai verbos (lemas no infinitivo) de um texto.

    Parâmetros:
        texto (str): Texto a analisar

    Retorna:
        list: Lista de lemas dos verbos

    Exemplo:
        >>> extrair_verbos("O cliente comprou e recomendou o produto")
        ['comprar', 'recomendar']
    """
    if texto is None:
        return []

    # Processar texto com o modelo spaCy
    doc = nlp_spacy(str(texto))

    # Filtrar apenas verbos (VERB), retornar lemas em minúsculas
    verbos = [token.lemma_.lower() for token in doc if token.pos_ == "VERB"]

    return verbos


# === APLICAR NO TEXTO DE VALIDAÇÃO ===
# Não modifique os nomes das variáveis abaixo

verbos_q11 = extrair_verbos(TEXTO_VALIDACAO_Q11)
num_verbos_q11 = len(verbos_q11)


In [ ]:
# Validação da Q11 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q11: extrair_verbos")
print("=" * 60)
print(f"\nTexto original (índice 53):")
print(f"  '{TEXTO_VALIDACAO_Q11}'")
print(f"\nVerbos extraídos ({num_verbos_q11}):")
print(f"  {verbos_q11}")
print(f"\nEstatísticas:")
print(f"  Total de verbos: {num_verbos_q11}")
print(f"  Verbos únicos: {len(set(verbos_q11))}")

VALIDAÇÃO Q11: extrair_verbos

Texto original (índice 53):
  'Gostei do produto! Valeu a pena a compra. Atendeu ao objetivo para o qual foi adquirido!'

Verbos extraídos (4):
  ['gostei', 'valer', 'atendeu', 'adquirir']

Estatísticas:
  Total de verbos: 4
  Verbos únicos: 4


---

## Questão 12: Função Integrada Final (8 pontos)

Implemente a função `criar_pipeline_preprocessamento` que retorna um **dicionário** com todas as funções de pré-processamento e análise, permitindo fácil reutilização.

O dicionário deve conter as seguintes chaves:
- `'limpar'`: referência para a função `limpar_texto`
- `'tokenizar'`: referência para a função `tokenizar`
- `'remover_stopwords'`: referência para a função `remover_stopwords`
- `'aplicar_stemming'`: referência para a função `aplicar_stemming`
- `'preprocessar'`: referência para a função `preprocessar_texto`
- `'processar_lote'`: referência para a função `processar_lote`
- `'analisar_frequencia'`: referência para a função `analisar_frequencia`
- `'extrair_pos_tags'`: referência para a função `extrair_pos_tags`
- `'extrair_substantivos'`: referência para a função `extrair_substantivos`
- `'extrair_adjetivos'`: referência para a função `extrair_adjetivos`
- `'extrair_verbos'`: referência para a função `extrair_verbos`

**IMPORTANTE:** Esta função será usada para importar o pipeline no projeto final!

**Retorno:**
- `dict`: Dicionário com as funções do pipeline

In [ ]:
def criar_pipeline_preprocessamento() -> dict:
    """
    Cria dicionário com todas as funções de pré-processamento e análise.

    Este pipeline será reutilizado no projeto final do chatbot.

    Retorna:
        dict: Dicionário com funções do pipeline

    Exemplo:
        >>> pipeline = criar_pipeline_preprocessamento()
        >>> pipeline['preprocessar']('Texto de teste')
        'text test'
        >>> pipeline['extrair_substantivos']('O produto é bom')
        ['produto']
    """
    pipeline = {
        "limpar": limpar_texto,
        "tokenizar": tokenizar,
        "remover_stopwords": remover_stopwords,
        "aplicar_stemming": aplicar_stemming,
        "preprocessar": preprocessar_texto,
        "processar_lote": processar_lote,
        "analisar_frequencia": analisar_frequencia,
        "extrair_pos_tags": extrair_pos_tags,
        "extrair_substantivos": extrair_substantivos,
        "extrair_adjetivos": extrair_adjetivos,
        "extrair_verbos": extrair_verbos,
    }

    return pipeline


# === CRIAR PIPELINE E TESTAR ===
# Não modifique os nomes das variáveis abaixo

pipeline_final_q12 = criar_pipeline_preprocessamento()

# Testar usando o pipeline
texto_teste_q12 = "O PRODUTO chegou muito rápido!!! Excelente qualidade."
resultado_nltk_q12 = pipeline_final_q12["preprocessar"](texto_teste_q12)
resultado_spacy_q12 = pipeline_final_q12["extrair_adjetivos"](texto_teste_q12)


In [ ]:
# Validação da Q12 - Execute para verificar sua implementação
print("=" * 60)
print("VALIDAÇÃO Q12: criar_pipeline_preprocessamento")
print("=" * 60)

# Verificar estrutura do pipeline
print(f"\nEstrutura do pipeline:")
print(f"  Tipo: {type(pipeline_final_q12).__name__}")
print(f"  Número de funções: {len(pipeline_final_q12)}")

# Verificar se todas as funções estão presentes
print(f"\nVerificação de funções:")
chaves_esperadas = [
    'limpar', 'tokenizar', 'remover_stopwords', 'aplicar_stemming',
    'preprocessar', 'processar_lote', 'analisar_frequencia',
    'extrair_pos_tags', 'extrair_substantivos', 'extrair_adjetivos', 'extrair_verbos'
]
for chave in chaves_esperadas:
    existe = chave in pipeline_final_q12
    callable_ok = callable(pipeline_final_q12.get(chave)) if existe else False
    status = "✓" if (existe and callable_ok) else "✗"
    print(f"  {status} {chave:25s}: existe={existe}, callable={callable_ok}")

# Teste funcional
print(f"\nTeste funcional:")
print(f"  Entrada: '{texto_teste_q12}'")
print(f"  NLTK preprocessar: '{resultado_nltk_q12}'")
print(f"  spaCy adjetivos: {resultado_spacy_q12}")

VALIDAÇÃO Q12: criar_pipeline_preprocessamento

Estrutura do pipeline:
  Tipo: dict
  Número de funções: 11

Verificação de funções:
  ✓ limpar                   : existe=True, callable=True
  ✓ tokenizar                : existe=True, callable=True
  ✓ remover_stopwords        : existe=True, callable=True
  ✓ aplicar_stemming         : existe=True, callable=True
  ✓ preprocessar             : existe=True, callable=True
  ✓ processar_lote           : existe=True, callable=True
  ✓ analisar_frequencia      : existe=True, callable=True
  ✓ extrair_pos_tags         : existe=True, callable=True
  ✓ extrair_substantivos     : existe=True, callable=True
  ✓ extrair_adjetivos        : existe=True, callable=True
  ✓ extrair_verbos           : existe=True, callable=True

Teste funcional:
  Entrada: 'O PRODUTO chegou muito rápido!!! Excelente qualidade.'
  NLTK preprocessar: 'produt cheg rápid excel qual'
  spaCy adjetivos: ['rápido', 'excelente']


---

## Resumo das Funções Implementadas

| Função | Descrição | Uso no Chatbot |
|--------|-----------|----------------|
| **NLTK - Pré-processamento** | | |
| `limpar_texto` | Limpeza básica | Pré-processamento de mensagens |
| `tokenizar` | Tokenização NLTK | Análise de tokens |
| `remover_stopwords` | Remove stopwords | Redução de ruído |
| `aplicar_stemming` | Stemming RSLP | Normalização |
| `preprocessar_texto` | Pipeline completo | **Principal** - usado em todas as etapas |
| `processar_lote` | Processamento em lote | Treinar classificador |
| `analisar_frequencia` | Contagem de termos | Análise exploratória |
| **spaCy - Análise Linguística** | | |
| `extrair_pos_tags` | Classes gramaticais | Análise morfossintática |
| `extrair_substantivos` | Substantivos (lemas) | Identificar tópicos/produtos |
| `extrair_adjetivos` | Adjetivos (lemas) | **Análise de sentimento** |
| `extrair_verbos` | Verbos (lemas) | Identificar ações do cliente |
| **Integração** | | |
| `criar_pipeline_preprocessamento` | Exporta funções | Integração com outras entregas |

---

## Checklist de Entrega

Antes de enviar, verifique:

- [ ] Todas as funções estão implementadas
- [ ] Todas as células foram executadas em ordem
- [ ] Os nomes das funções não foram alterados
- [ ] O código está comentado
- [ ] Os testes básicos passaram

---

## Submissão

1. Verifique se todas as células foram executadas em ordem
2. Confira se não há erros de execução
3. Limpe as células de saída (Edit > Clear all outputs)
4. Salve o notebook (Ctrl+S ou File > Save)
5. Faça download do arquivo .ipynb
6. Submeta no Moodle até a data limite

**Envie este notebook via Moodle até a data limite.**